CNN 및 5개의 Pre-Trained Model을 돌린 후 **val-acc가 최대**일 때를 기준으로 성능 평가표를 만든다

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/KSEF/images.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: images/images/validation/surprise/31961.jpg  
  inflating: __MACOSX/images/images/validation/surprise/._31961.jpg  
  inflating: images/images/validation/surprise/29690.jpg  
  inflating: __MACOSX/images/images/validation/surprise/._29690.jpg  
  inflating: images/images/validation/surprise/22921.jpg  
  inflating: __MACOSX/images/images/validation/surprise/._22921.jpg  
  inflating: images/images/validation/surprise/22666.jpg  
  inflating: __MACOSX/images/images/validation/surprise/._22666.jpg  
  inflating: images/images/validation/surprise/34004.jpg  
  inflating: __MACOSX/images/images/validation/surprise/._34004.jpg  
  inflating: images/images/validation/surprise/6877.jpg  
  inflating: __MACOSX/images/images/validation/surprise/._6877.jpg  
  inflating: images/images/validation/surprise/34986.jpg  
  inflating: __MACOSX/images/images/validation/surprise/._34986.jpg  
  inflating: images/images/validation/surprise/4912.jpg  
  inf

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.applications import ResNet50V2 as ResNet
from tensorflow.keras.applications.xception import Xception as xcept
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score
from keras.layers import Activation
from keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.applications import DenseNet201

In [ ]:
batch_size = 64
train_dir = "/content/images/train"
test_dir = "/content/images/validation"
num_classes = 7

In [ ]:
datagen = ImageDataGenerator(rescale=1 / 255)


train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)


test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [ ]:
def f1_metric(y_true, y_pred):
    y_pred = tf.round(y_pred)
    f1 = 2 * tf.reduce_sum(y_true * y_pred) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1e-10)
    return f1

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad','surprise']

def cm(y_true, y_pred):
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Plot the confusion matrix as a heatmap
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=labels, yticklabels=labels)

    # Set labels
    plt.xlabel('Predicted')
    plt.ylabel('True')

    # Show the plot
    plt.show()

<h1>Base Model: <strong>CNN</strong></h1>

<h1>Enhanced Model: <strong>Xception(Best so far), ResNet-50, MobileNetV2, InceptionV3, DenseNet</h1></strong>

In [ ]:
# CNN

model = Sequential()

#1st CNN layer
model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (224,224,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#2nd CNN layer
model.add(Conv2D(128,(5,5),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#3rd CNN layer
model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#4th CNN layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

opt = Adam(lr = 0.001)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy', f1_metric])

"""
history = model.fit(train_generator,
          epochs=50,
          validation_data=test_generator,
          callbacks=[early_stopping])
"""
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization (Batch  (None, 224, 224, 64)      256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 224, 224, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 112, 112, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 128)     2

In [ ]:
# Resnet

resnet = ResNet(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

x = resnet.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.models.Model(inputs=resnet.input, outputs=predictions)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy', f1_metric])

history = model.fit(train_generator,
          epochs=50,
          validation_data=test_generator,
          callbacks=[early_stopping])

In [ ]:
# Xception

# Define a callback to save the model with the best validation accuracy
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

# Rest of your code
xception = xcept(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

x = xception.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.models.Model(inputs=xception.input, outputs=predictions)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy', f1_metric])

history = model.fit(train_generator,
          epochs=50,
          validation_data=test_generator,
          callbacks=[early_stopping, model_checkpoint])


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

class_labels = ['angry','disgust','fear','happy','neutral','sad','surprise']

predictions = model.predict_generator(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
confusion_mat = confusion_matrix(true_classes, predicted_classes)

<ipython-input-11-45011621da0c>:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(test_generator)


In [ ]:
# Inception V3

num_classes = 7

# Define a callback to save the model with the best validation accuracy
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

# Load InceptionV3 with pre-trained weights
inception = InceptionV3(include_top=False, weights='imagenet', input_shape=(299, 299, 3))

"""
# Freeze the convolutional base
for layer in inception.layers:
    layer.trainable = False
"""

# Build the model on top of InceptionV3
x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

predictions = Dense(num_classes, activation='softmax')(x)
model = tf.keras.models.Model(inputs=inception.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy', f1_metric])

# Assuming you have 'train_generator' and 'test_generator' defined
# Also, you need to have 'EarlyStopping' imported from tensorflow.keras.callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(train_generator,
                    epochs=50,
                    validation_data=test_generator,
                    callbacks=[early_stopping, model_checkpoint])


Epoch 1/50
451/451 [==============================] - 256s 493ms/step - loss: 1.4940 - accuracy: 0.3984 - f1_metric: 0.2899 - val_loss: 20.0542 - val_accuracy: 0.3670 - val_f1_metric: 0.3466
Epoch 2/50
451/451 [==============================] - 218s 483ms/step - loss: 1.1875 - accuracy: 0.5502 - f1_metric: 0.4702 - val_loss: 1.1739 - val_accuracy: 0.5566 - val_f1_metric: 0.4772
Epoch 3/50
451/451 [==============================] - 218s 483ms/step - loss: 1.0651 - accuracy: 0.6072 - f1_metric: 0.5511 - val_loss: 1.2367 - val_accuracy: 0.5776 - val_f1_metric: 0.5600
Epoch 4/50
451/451 [==============================] - 218s 483ms/step - loss: 0.9765 - accuracy: 0.6402 - f1_metric: 0.6009 - val_loss: 1.1791 - val_accuracy: 0.5923 - val_f1_metric: 0.5568
Epoch 5/50
451/451 [==============================] - 218s 483ms/step - loss: 0.8933 - accuracy: 0.6678 - f1_metric: 0.6369 - val_loss: 1.0844 - val_accuracy: 0.6047 - val_f1_metric: 0.5914
Epoch 6/50
451/451 [=============================

In [ ]:
# DenseNet

# Assuming you have the number of classes defined as 'num_classes'
num_classes = 7  # Replace with the actual number of classes in your classification task

# Define a callback to save the model with the best validation accuracy
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

# Load DenseNet201 with pre-trained weights
densenet = DenseNet201(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

"""
# Freeze the convolutional base
for layer in densenet.layers:
    layer.trainable = False
"""

# Build the model on top of DenseNet201
x = densenet.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

predictions = Dense(num_classes, activation='softmax')(x)
model = tf.keras.models.Model(inputs=densenet.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy', f1_metric])

# Assuming you have 'train_generator' and 'test_generator' defined
# Also, you need to have 'EarlyStopping' imported from tensorflow.keras.callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(train_generator,
                    epochs=50,
                    validation_data=test_generator,
                    callbacks=[early_stopping, model_checkpoint])

In [ ]:
# MobileNet

# Assuming you have the number of classes defined as 'num_classes'
num_classes = 7  # Replace with the actual number of classes in your classification task

# Define a callback to save the model with the best validation accuracy
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

# Load MobileNetV2 with pre-trained weights
mobilenet_v2 = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

"""
# Freeze the convolutional base
for layer in mobilenet_v2.layers:
    layer.trainable = False
"""

# Build the model on top of MobileNetV2
x = mobilenet_v2.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

predictions = Dense(num_classes, activation='softmax')(x)
model = tf.keras.models.Model(inputs=mobilenet_v2.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy', f1_metric])

# Assuming you have 'train_generator' and 'test_generator' defined
# Also, you need to have 'EarlyStopping' imported from tensorflow.keras.callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(train_generator,
                    epochs=50,
                    validation_data=test_generator,
                    callbacks=[early_stopping, model_checkpoint])